# Lab 8 - Back to the Digital Twin

In [ ]:

from dotenv import load_dotenv
import os
import requests
from pypdf import PdfReader
import gradio as gr
from agents import Agent, Runner, function_tool
load_dotenv(override=True)


In [2]:
pushover_user = os.getenv("PUSHOVER_USER")
pushover_token = os.getenv("PUSHOVER_TOKEN")
pushover_url = "https://api.pushover.net/1/messages.json"

In [3]:
def push(message: str):
    payload = {"user": pushover_user, "token": pushover_token, "message": message}
    requests.post(pushover_url, data=payload)
    return {"status": "success"}

In [4]:
@function_tool
def record_user_details(email: str, name: str = "Name not provided", notes: str = "not provided") -> str:
    """
    Record that a user is interested in being in touch and provided an email address
    Args:
        email: The email address of the user
        name: The name of the user, optionally
        notes: if provided, any additional information about the conversation to give context on why they're getting in touch
    """

    push(f"Recording interest from {name} with email {email} and notes {notes}")
    return "Recorded"

In [5]:
@function_tool
def record_unknown_question(question: str) -> str:
    """
    Record that a user asked a question that you couldn't answer based on your knowledge.
    This will be recorded so that the knowledge can be updated later.
    
    Args:
        question: The question that was asked
    """
    push(f"Recording {question} asked that I couldn't answer")
    return "This question has been recorded. Please ask the user for their email address and record it so their question can be answered in the future."

In [6]:
reader = PdfReader("../week2/me/linkedin.pdf")
linkedin = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin += text

with open("../week2/me/summary.txt", "r", encoding="utf-8") as f:
    summary = f.read()

name = "Ed Donner"

In [7]:
instructions = f"""
You are acting as {name}. You are answering questions on {name}'s website,
particularly questions related to {name}'s career, background, skills and experience.
Your responsibility is to represent {name} for interactions on the website as faithfully as possible.
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions.
Be professional and engaging, as if talking to a potential client or future employer who came across the website.
If you don't know the answer to any question, use your record_unknown_question tool to record the question that you couldn't answer,
even if it's about something trivial or unrelated to career.
Then ask them for their email address and record it, so that their question can be answered in the future.
If the user is engaging in discussion, try to steer them towards getting in touch via email; ask for their email 
and record it using your record_user_details tool.
If they ask how to get in touch, ask them for their email address and record it using your record_user_details tool.

## Summary of {name}
{summary}

## LinkedIn Profile of {name}
{linkedin}

With this context, please chat with the user, always staying in character as {name}.
"""

In [8]:
agent = Agent(name="Digital Twin", model="gpt-4.1-mini", instructions=instructions, tools=[record_unknown_question, record_user_details])

In [9]:
async def chat(message, history):
    messages = [{"role": prior["role"], "content": prior["content"]} for prior in history]  
    messages += [{"role": "user", "content": message}]
    response = await Runner.run(agent, messages)
    return response.final_output

In [ ]:
gr.ChatInterface(chat, type="messages").launch()

## And now.. a fancier User Interface

Courtesy of Claude 4.0 Sonnet

In [ ]:
PRIMARY_COLOR = "#2563eb"
SECONDARY_COLOR = "#f8fafc"
ACCENT_COLOR = "#3b82f6"
TEXT_COLOR = "#1e293b"
BORDER_COLOR = "#e2e8f0"

EXAMPLE_QUESTIONS = [
    "What are your top accomplishments?",
    "Tell me about your most exciting project",
    "How do I get in touch with you?",
    "Do you like cheese?",
]

custom_css = f"""
/* Define CSS variables for theming */
:root {{
    --primary-color: {PRIMARY_COLOR};
    --accent-color: {ACCENT_COLOR};
    --card-bg: #ffffff;
    --text-primary: #1e293b;
    --text-secondary: #64748b;
    --border-color: #e2e8f0;
    --example-bg: #f8fafc;
}}

.dark {{
    --card-bg: #1e293b;
    --text-primary: #f1f5f9;
    --text-secondary: #94a3b8;
    --border-color: #334155;
    --example-bg: #334155;
}}

/* Main container styling */
.gradio-container {{
    max-width: 900px !important;
    margin: 0 auto !important;
}}

/* Header styling */
.header-container {{
    text-align: center;
    padding: 2rem 1rem 1.5rem 1rem;
    background: var(--card-bg);
    border-radius: 12px;
    margin-bottom: 1.5rem;
    box-shadow: 0 4px 6px -1px rgba(0, 0, 0, 0.1);
    border: 1px solid var(--border-color);
}}

.main-title {{
    font-size: 2.5rem;
    font-weight: 700;
    margin-bottom: 0.5rem;
    color: white !important;
    display: inline-block;
}}

.subtitle {{
    font-size: 1.1rem;
    color: var(--text-secondary);
    margin-bottom: 1rem;
}}

/* Example questions styling */
.examples-container {{
    background: var(--card-bg);
    border-radius: 12px;
    padding: 1.5rem;
    margin-bottom: 1.5rem;
    box-shadow: 0 2px 4px rgba(0, 0, 0, 0.05);
    border: 1px solid var(--border-color);
}}

.examples-title {{
    font-size: 1.1rem;
    font-weight: 600;
    color: var(--text-primary);
    margin-bottom: 1rem;
}}

.example-btn {{
    background: var(--example-bg) !important;
    color: var(--primary-color) !important;
    border: 1px solid var(--border-color) !important;
    border-radius: 20px !important;
    padding: 0.5rem 1rem !important;
    margin: 0.25rem !important;
    font-size: 0.9rem !important;
    cursor: pointer !important;
    transition: all 0.2s ease !important;
}}

.example-btn:hover {{
    background: var(--primary-color) !important;
    color: white !important;
    transform: translateY(-1px) !important;
    box-shadow: 0 2px 4px rgba(0, 0, 0, 0.1) !important;
}}

/* Chat interface improvements */
[data-testid="chatbot"] {{
    background: var(--card-bg) !important;
    border-radius: 12px !important;
    border: 1px solid var(--border-color) !important;
}}

/* User message styling */
[data-testid="chatbot"] .message-wrap.user {{
    background: var(--primary-color) !important;
    color: white !important;
}}

/* Bot message styling */
[data-testid="chatbot"] .message-wrap.bot {{
    background: var(--example-bg) !important;
    color: var(--text-primary) !important;
    border-left: 3px solid var(--primary-color) !important;
}}

/* Footer styling */
.footer {{
    text-align: center;
    padding: 1rem;
    color: var(--text-secondary);
    font-size: 0.9rem;
    margin-top: 1rem;
}}
footer{{display:none !important}}
"""

with gr.Blocks(css=custom_css, title="My Digital Twin", theme=gr.themes.Soft()) as interface:
    
    # Header section
    with gr.Row(elem_classes="header-container"):
        gr.HTML(f"""
            <div class="main-title">{name}'s Digital Twin</div>
            <div class="subtitle">Ask me anything about my professional background, skills, and experience</div>
        """)
    
    # Example questions section
    with gr.Row(elem_classes="examples-container"):
        gr.HTML('<div class="examples-title">💡 Try asking me:</div>')
        with gr.Row():
            example_buttons = []
            for question in EXAMPLE_QUESTIONS:
                btn = gr.Button(question, elem_classes="example-btn", size="sm")
                example_buttons.append(btn)
    
    # Main chat interface
    chatbot_interface = gr.ChatInterface(
        chat,
        type="messages",
        chatbot=gr.Chatbot(
            height=500,
            placeholder=f"👋 Hi! I'm {name}'s digital twin. Ask me about my experience, skills, or career journey!",
            type="messages"
        ),
        textbox=gr.Textbox(
            placeholder="Ask me about my background, skills, projects, or experience...",
            container=False,
            scale=7
        )
    )
    
    for btn in example_buttons:
        btn.click(lambda q: q, inputs=[btn], outputs=[chatbot_interface.textbox])
    
    gr.HTML(f"<div class='footer'>Powered by {name}'s digital twin</div>")
    

interface.launch()

# Conclusion

### Assignments

1. Separate Digital Twin into Module
2. With help from an LLM, give it a unique, fresh coat of CSS - a look for you
3. Add in your improved prompts and rate limiting from last time
4. Add another tool to improve expertise

### Stretch assignments - any of these:

1. Digital twin: Add voice
2. Digital twin: Use a different observability platform, like MLFlow
3. Deep Research: add clarifying questions
4. Operator: Update the Operator to use Playwright over CDP and your full Chrome so you can collaborate with your Operator


### Upcoming sessions

AMA with Jon Krohn at 5pm on Wednesday

Office hours on Thursday